# Starting to explore daily train data

In [14]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import time

# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data
from fetchWeatherData import fetch_weather_data

date = datetime.date.today() - datetime.timedelta(days=1)
unix_time = time.mktime(date.timetuple())

# Fetch trains for a given station
q = (
    """ 
    {
    """
      f'  trainsByDepartureDate(departureDate: \"{date}\",'
    """
        take: 5, 
        where: {
            and: [
                {or: [{deleted: {unequals: null}}, 
                {deleted: {equals: false}}]}, 
                {cancelled: {equals: false}}, 
                {operator: {shortCode: {equals: "vr"}}}
                ]  
        }
      ) 
      {
        trainNumber
        departureDate
        commuterLineid
        trainType {
          name
          trainCategory {
            name
          }
        }
        timeTableRows {
        differenceInMinutes
          station {
            shortCode
            location
          }
        }
      }
    }
    """
    )

result = fetch_train_data(q)

recs = result['data']['trainsByDepartureDate']
recs

[{'trainNumber': 1,
  'departureDate': '2021-10-24',
  'commuterLineid': '',
  'trainType': {'name': 'IC', 'trainCategory': {'name': 'Long-distance'}},
  'timeTableRows': [{'differenceInMinutes': 3,
    'station': {'shortCode': 'HKI', 'location': [24.94125, 60.172096]}},
   {'differenceInMinutes': 3,
    'station': {'shortCode': 'PSL', 'location': [24.933521, 60.19869]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'PSL', 'location': [24.933521, 60.19869]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'KÄP', 'location': [24.946033, 60.22017]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'KÄP', 'location': [24.946033, 60.22017]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'OLK', 'location': [24.967676, 60.229]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'OLK', 'location': [24.967676, 60.229]}},
   {'differenceInMinutes': 4,
    'station': {'shortCode': 'PMK', 'location': [24.993288, 60.242214]}},
   {'diffe

In [8]:
df = pd.json_normalize(recs)
all_stops = pd.json_normalize(df['timeTableRows'])
df = df.drop(['timeTableRows'], axis=1)

weather_data = {}

def get_weather(tag, lat=0, lon=0):
    if tag not in weather_data:
        print("FETCHING DATA..")
        print(unix_time)
        print(lat)
        print(lon)
        weather_data[tag] = fetch_weather_data(unix_time, lat, lon)

def extract_time_tables(train_stops_):
    train_stops = pd.json_normalize(train_stops_)
    to_return = train_stops['differenceInMinutes'].sum()
    train_stops = train_stops.drop(['differenceInMinutes'], axis=1)

    train_stops.apply(lambda x : get_weather(x['station.shortCode'], x['station.location'][1], x['station.location'][0]), axis=1)
        
    for stop in train_stops['station.shortCode'].unique():
        if stop not in df.columns:
            df[stop] = None
        
        df[stop][train_stops_.name] = weather_data[stop]
    
    return to_return

how_much_late = all_stops.apply(extract_time_tables, axis=1)
df

FETCHING DATA..
1635109200.0
60.172096
24.94125


Exception: Query failed to run with a 404

In [30]:
print(fetch_weather_data(1635109200.0, 60.172096, 8))

Exception: Query failed to run with a 404